In [1]:
import tensorflow as tf


2025-03-15 13:46:29.451337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742046389.459342  296080 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742046389.461762  296080 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742046389.469765  296080 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742046389.469774  296080 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742046389.469776  296080 computation_placer.cc:177] computation placer alr

In [2]:
ip_bbox = tf.constant([[0.31654677, 0.5151515  ,0.58992803, 0.9015151 ],
                       [0.28057554, 0.5 ,       0.6906475 , 0.90909094],
                       [0.,         0. ,        0.     ,    0.        ],
                       ], dtype=tf.float32)
ip_bbox, ip_bbox.shape

I0000 00:00:1742046393.326725  296080 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7213 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6


(<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[0.31654677, 0.5151515 , 0.58992803, 0.9015151 ],
        [0.28057554, 0.5       , 0.6906475 , 0.90909094],
        [0.        , 0.        , 0.        , 0.        ]], dtype=float32)>,
 TensorShape([3, 4]))

In [3]:
ip_bbox[..., 0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.31654677, 0.28057554, 0.        ], dtype=float32)>

In [4]:
all_zero_rows = tf.reduce_all(ip_bbox == 0, axis=1) 
zero_row_indices = tf.where(all_zero_rows)[:, 0]  # Get indices from tf.where
all_zero_rows, zero_row_indices

(<tf.Tensor: shape=(3,), dtype=bool, numpy=array([False, False,  True])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([2])>)

In [5]:
tf.where(all_zero_rows)[:, 0]

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([2])>

In [6]:
import tensorflow as tf
import numpy as np

def find_zero_row_indices(arr):
    """
    Finds the indices of rows that are all zeros in a TensorFlow tensor.

    Args:
        arr: The TensorFlow tensor.

    Returns:
        A TensorFlow tensor of indices where the rows are all zeros.
    """
    all_zero_rows = tf.reduce_all(arr == 0, axis=1) #  find all rows that are all zeros
    zero_row_indices = tf.where(all_zero_rows)[:, 0]  # Get indices from tf.where
    return zero_row_indices

def drop_rows_by_indices(arr, indices_to_drop):
    """
    Drops rows from a TensorFlow tensor based on the given indices.

    Args:
        arr: The TensorFlow tensor.
        indices_to_drop: A TensorFlow tensor of indices to drop.

    Returns:
        A new TensorFlow tensor with the specified rows removed.
    """
    # Create a boolean mask where True means "keep" and False means "drop"
    mask = tf.ones(tf.shape(arr)[0], dtype=tf.bool)
    
    # Use tf.tensor_scatter_nd_update to set the mask to False at the specified indices
    updates = tf.fill(tf.shape(indices_to_drop), False) # create a vector of False
    indices = tf.expand_dims(indices_to_drop, axis=1) # add a dimension
    mask = tf.tensor_scatter_nd_update(mask, indices, updates) # set the mask to False

    # Use boolean indexing to select the rows to keep
    return tf.boolean_mask(arr, mask)

# Example Usage (to demonstrate the fix):
ip_bbox = tf.constant([[0.,         0. ,        0.     ,    0.        ],
                        [0.31654677, 0.5151515  ,0.58992803, 0.9015151 ],
                       [0.28057554, 0.5 ,       0.6906475 , 0.90909094]
], dtype=tf.float32)



zero_indices = find_zero_row_indices(ip_bbox)
print(f"Indices of all-zero rows: {zero_indices}")

filtered_bbox = drop_rows_by_indices(ip_bbox, zero_indices)
print(f"Filtered bounding boxes:\n{filtered_bbox}")


Indices of all-zero rows: [0]
Filtered bounding boxes:
[[0.31654677 0.5151515  0.58992803 0.9015151 ]
 [0.28057554 0.5        0.6906475  0.90909094]]


In [7]:
def iou_loss(y_true, y_pred):  # Assuming y_true and y_pred are (batch_size, 4)
    """x_min, y_min, x_max, y_max
    """
    y_true_zero_indices= find_zero_row_indices(y_true)
    y_true = drop_rows_by_indices(y_true, y_true_zero_indices)
    y_pred = drop_rows_by_indices(y_pred, y_true_zero_indices)

    x_min_true = y_true[..., 0]
    y_min_true = y_true[..., 1]
    x_max_true = y_true[..., 2]
    y_max_true = y_true[..., 3]

    x_min_pred = y_pred[..., 0]
    y_min_pred = y_pred[..., 1]
    x_max_pred = y_pred[..., 2]
    y_max_pred = y_pred[..., 3]

    area_true = (x_max_true - x_min_true) * (y_max_true - y_min_true)
    area_pred = (x_max_pred - x_min_pred) * (y_max_pred - y_min_pred)

    x_intersect = tf.maximum(x_min_true, x_min_pred)
    y_intersect = tf.maximum(y_min_true, y_min_pred)
    x_max_intersect = tf.minimum(x_max_true, x_max_pred)
    y_max_intersect = tf.minimum(y_max_true, y_max_pred)

    area_intersect = tf.maximum(0.0, x_max_intersect - x_intersect) * tf.maximum(0.0, y_max_intersect - y_intersect) # avoid negative values
    iou = area_intersect / (area_true + area_pred - area_intersect + 1e-7)  # Add small epsilon for numerical stability
    return 1.0 - iou  # We want to *minimize* the loss


In [8]:
pred_ip_bbox = tf.constant([[0.28057554, 0.5 ,       0.6906475 , 0.90909094 ],
                       [0.28057554, 0.5 ,       0.6906475 , 0.90909094],
                       [0.28057554, 0.5 ,       0.6906475 , 0.90909094]
                   ], dtype=tf.float32)

y_trueip_bbox = tf.constant([[0.28057554, 0.5 ,       0.6906475 , 0.90909094 ],
                       [0, 0,0,0],
                       [0.28057554, 0.5 ,       0.6906475 , 0.90909094]
                   ], dtype=tf.float32)

In [9]:
iou_loss(y_trueip_bbox, pred_ip_bbox)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([5.9604645e-07, 5.9604645e-07], dtype=float32)>

In [10]:
def eff_iou_loss(y_true, y_pred):  # Assuming y_true and y_pred are (batch_size, 4)
    """x_min, y_min, x_max, y_max
    """
    # Create a mask for valid bounding boxes
    mask = tf.reduce_any(tf.not_equal(y_true, 0), axis=-1)
    mask = tf.cast(mask, dtype=tf.float32)

    x_min_true = y_true[..., 0]
    y_min_true = y_true[..., 1]
    x_max_true = y_true[..., 2]
    y_max_true = y_true[..., 3]

    x_min_pred = y_pred[..., 0]
    y_min_pred = y_pred[..., 1]
    x_max_pred = y_pred[..., 2]
    y_max_pred = y_pred[..., 3]

    area_true = (x_max_true - x_min_true) * (y_max_true - y_min_true)
    area_pred = (x_max_pred - x_min_pred) * (y_max_pred - y_min_pred)

    x_intersect = tf.maximum(x_min_true, x_min_pred)
    y_intersect = tf.maximum(y_min_true, y_min_pred)
    x_max_intersect = tf.minimum(x_max_true, x_max_pred)
    y_max_intersect = tf.minimum(y_max_true, y_max_pred)

    area_intersect = tf.maximum(0.0, x_max_intersect - x_intersect) * tf.maximum(0.0, y_max_intersect - y_intersect) # avoid negative values
    iou = area_intersect / (area_true + area_pred - area_intersect + 1e-7)  # Add small epsilon for numerical stability
    # loss = iou #1.0 - iou 
    # Apply mask and calculate mean loss
    # masked_loss = loss * mask
    loss = tf.boolean_mask(iou, mask)
    return 1- loss #tf.reduce_sum(masked_loss) / (tf.reduce_sum(mask) + 1e-7)

In [11]:
eff_iou_loss(y_trueip_bbox, pred_ip_bbox)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([5.9604645e-07, 5.9604645e-07], dtype=float32)>

In [12]:
# Create a mask for valid bounding boxes
mask = tf.reduce_any(tf.not_equal(ip_bbox, 0), axis=-1)
mask = tf.cast(mask, dtype=tf.float32)
mask 

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 1.], dtype=float32)>

In [13]:
def giou_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    
    # Assuming y_true and y_pred are (batch_size, num_classes, 4)
    
    # Create a mask for valid bounding boxes
    mask = tf.reduce_any(tf.not_equal(y_true, 0), axis=-1)
    mask = tf.cast(mask, dtype=tf.float32)
    
    # Calculate IoU
    intersect_mins = tf.maximum(y_true[..., :2], y_pred[..., :2])
    intersect_maxes = tf.minimum(y_true[..., 2:], y_pred[..., 2:])
    intersect_wh = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_area = (y_true[..., 2] - y_true[..., 0]) * (y_true[..., 3] - y_true[..., 1])
    pred_area = (y_pred[..., 2] - y_pred[..., 0]) * (y_pred[..., 3] - y_pred[..., 1])
    
    union_area = true_area + pred_area - intersect_area
    iou = intersect_area / (union_area + 1e-7)
    
    # Calculate GIoU
    enclose_mins = tf.minimum(y_true[..., :2], y_pred[..., :2])
    enclose_maxes = tf.maximum(y_true[..., 2:], y_pred[..., 2:])
    enclose_wh = tf.maximum(enclose_maxes - enclose_mins, 0.)
    enclose_area = enclose_wh[..., 0] * enclose_wh[..., 1]
    
    giou = iou - (enclose_area - union_area) / (enclose_area + 1e-7)
    loss = 1 - giou
    
    # Apply mask and calculate mean loss
    masked_loss = loss * mask
    return tf.reduce_sum(masked_loss) / (tf.reduce_sum(mask) + 1e-7)

In [16]:
def adapted_giou_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    # Create a mask for valid bounding boxes
    mask = tf.reduce_any(tf.not_equal(y_true, 0), axis=-1)
    mask = tf.cast(mask, dtype=tf.float32)
    
    # Calculate IoU
    # intersect_mins = tf.maximum(y_true[..., :2], y_pred[..., :2])
    # intersect_maxes = tf.minimum(y_true[..., 2:], y_pred[..., 2:])
    # intersect_wh = tf.maximum(intersect_maxes - intersect_mins, 0.)
    # intersect_area = intersect_wh[..., 0]* intersect_wh[..., 1]
    
    # true_area = (y_true[..., 2] - y_true[..., 0]) * (y_true[..., 3] - y_true[..., 1])
    # pred_area = (y_pred[..., 2] - y_pred[..., 0]) * (y_pred[..., 3] - y_pred[..., 1])
    
    # union_area = (true_area + pred_area) - intersect_area
    # iou = intersect_area / (union_area + 1e-7)
    x_min_true = y_true[..., 0]
    y_min_true = y_true[..., 1]
    x_max_true = y_true[..., 2]
    y_max_true = y_true[..., 3]

    x_min_pred = y_pred[..., 0]
    y_min_pred = y_pred[..., 1]
    x_max_pred = y_pred[..., 2]
    y_max_pred = y_pred[..., 3]

    area_true = (x_max_true - x_min_true) * (y_max_true - y_min_true)
    area_pred = (x_max_pred - x_min_pred) * (y_max_pred - y_min_pred)

    x_intersect = tf.maximum(x_min_true, x_min_pred)
    y_intersect = tf.maximum(y_min_true, y_min_pred)
    x_max_intersect = tf.minimum(x_max_true, x_max_pred)
    y_max_intersect = tf.minimum(y_max_true, y_max_pred)

    area_intersect = tf.maximum(0.0, x_max_intersect - x_intersect) * tf.maximum(0.0, y_max_intersect - y_intersect) # avoid negative values
    union_area = (area_true + area_pred - area_intersect + 1e-7) 
    iou = area_intersect / union_area # Add small epsilon for numerical stability

    # Calculate GIoU
    enclose_mins = tf.minimum(y_true[..., :2], y_pred[..., :2])
    enclose_maxes = tf.maximum(y_true[..., 2:], y_pred[..., 2:])
    enclose_wh = tf.maximum(enclose_maxes - enclose_mins, 0.)
    enclose_area =enclose_wh[..., 0] * enclose_wh[..., 1]
    
    giou = iou - ((enclose_area - union_area) / (enclose_area + 1e-7))

    giou_loss = tf.boolean_mask(giou, mask, axis=0)
    return 1-tf.math.reduce_mean (giou_loss)

In [20]:
pred_ip_bbox = tf.constant([[0.28057554, 0.5 ,       0.6906475 , 0.90909094 ],
                       [0.48057554, 0.5 ,       0.4906475 , 0.70909094],
                       [0.68057554, 0.5 ,       0.5906475 , 0.30909094]
                   ], dtype=tf.float32)

y_trueip_bbox = tf.constant([[0.28057554, 0.5 ,       0.6906475 , 0.90909094 ],
                       [0, 0,0,0],
                       [0.28057554, 0.5 ,       0.6906475 , 0.90909094]
                   ], dtype=tf.float32)

In [21]:
adapted_giou_loss(y_trueip_bbox, pred_ip_bbox)

<tf.Tensor: shape=(), dtype=float32, numpy=0.44883012771606445>